#  Analyze EGFD Accountability Data

E.Quinn  6/28/2018

In [543]:
import re
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import struct
from datetime import datetime
import datetime

In [544]:
pd.set_option('display.max_rows', 3000)

## Dictionary for looking up name by page

In [545]:
FF_name = {1: 'FF Andrade', 2: 'FF Archambault', 3: 'Lt Babcock', 4: 'Lt Bailey', 5: 'Lt Beaudreau', \
    6: 'FF Campbell', 7: 'FF Columbier', 8: 'Prob-FF Crute', 9: 'FF DeLuca', 10: 'FF Forte', \
    11: 'Lt Gardner', 12: 'Prob-FF Gorman', 13: 'Lt Grady', 14: 'Lt Greene', \
    15: 'Lt Hall', 16: 'FF Howard', 17: 'Lt Jones', 18: 'FF King', 19: 'FF Lang Jr.', 20: 'Prob-FF Lavallee', \
    21: 'FF Marsh', 22: 'Lt Matola, Jr.', 23: 'FF McKeon', 24: 'Capt Mears', 25: 'Lt Monaghan', \
    26: 'Capt Montville', 27: "FF O'Donnell", 28: 'Prob-FF Perry', 29: 'Lt Perry', 30: 'Prob-FF Preston', \
    31: 'Lt Purcell', 32: 'Lt Richardson', 33: 'FF Snowling', 34: 'FF Stabile', 35: 'FF Szerlag', 36: 'Lt Warner III'}                 

## Dictionary for looking up page by name

In [546]:
FF_ID = {'FF Andrade': 1,'FF Archambault': 2,'Lt Babcock': 3,'Lt Bailey': 4,'Lt Beaudreau': 5,'FF Campbell': 6, \
         'FF Columbier': 7,'Prob-FF Crute': 8,'FF DeLuca': 9,'FF Forte': 10,'Lt Gardner': 11,'Prob-FF Gorman': 12, \
         'Lt Grady': 13,'Lt Greene': 14,'Lt Hall': 15,'FF Howard': 16,'Lt Jones': 17,'FF King': 18,'FF Lang Jr.': 19, \
         'Prob-FF Lavallee': 20,'FF Marsh': 21,'Lt Matola, Jr.': 22,'FF McKeon': 23,'Capt Mears': 24,'Lt Monaghan': 25, \
         'Capt Montville': 26,"FF O'Donnell": 27,'Prob-FF Perry': 28,'Lt Perry': 29,'Prob-FF Preston': 30,  \
         'Lt Purcell': 31,'Lt Richardson': 32,'FF Snowling': 33,'FF Stabile': 34,'FF Szerlag': 35,'Lt Warner III': 36}                 


## Dictionary for looking up designation by name

In [547]:
FF_desig = {'FF Andrade': 'FFC2','FF Archambault': 'FFA1','Lt Babcock': 'OFD2','Lt Bailey': 'OFA2', \
        'Lt Beaudreau': 'OFB3', 'FF Campbell': 'FFB1', 'FF Columbier':'FFB4', 'Prob-FF Crute':'FFA5', \
            'FF DeLuca': 'FFC4', 'FF Forte': 'OFB2', 'Lt Gardner': 'FFD5',   \
          'Lt Grady': 'OFB4','Lt Greene': 'OFC4','Lt Hall': 'OFD1', 'Prob-FF Gorman': 'FFB3', 'FF Howard':'OFC2', \
            'Lt Jones': 'FFC1', 'FF King':'FFD3', 'FF Lang Jr.':'FFD3', 'Prob-FF Lavallee': 'FFD4', \
        'FF Marsh': 'FFD1','Lt Matola, Jr.': 'OFD4','FF McKeon': 'FFC5','Capt Mears': 'OFC1','Lt Monaghan': 'OFA4', \
        'Capt Montville': 'OFA3',"FF O'Donnell": 'FFA2','Prob-FF Perry': 'FFB5','Lt Perry': 'OFB1', \
        'Prob-FF Preston': 'FFA3','Lt Purcell': 'OFC3','Lt Richardson': 'OFA1','FF Snowling': 'FFB2', \
        'FF Stabile': 'FFD2', 'FF Szerlag': 'FFC3', 'Lt Warner III': 'OFD3'}                 


## Dictionary for looking up name by designation

In [548]:
desig_FF = {'FFC2' : 'FF Andrade','FFA1' : 'FF Archambault','OFD2' : 'Lt Babcock','OFA2' : 'Lt Bailey',  \
           'OFB3': 'Lt Beaudreau', 'FFA4': 'FF Campbell', 'FFB1': 'FF Columbier','FFB4': 'Prob-FF Crute', \
            'FFA5': 'FF DeLuca','FFC4': 'FF Forte', 'OFB2': 'Lt Gardner', 'FFD5': 'Prob-FF Gorman', \
            'OFB4': 'Lt Grady','OFC4': 'Lt Greene','OFD1': 'Lt Hall', 'FFB3': 'FF Howard', 'OFC2': 'Lt Jones', \
           'FFC1': 'FF King','FFD3': 'FF Lang Jr.', 'FFD4':'Prob-FF Lavallee', 'FFC2' : 'FF Andrade', \
            'FFA1' : 'FF Archambault','OFD2' : 'Lt Babcock','OFA2' : 'Lt Bailey','FFA2': "FF O'Donnell", \
            'OFA3': 'Capt Montville','FFB5': 'Prob-FF Perry','OFB1': 'Lt Perry', 'OFD4': 'Lt Matola, Jr.', \
            'FFD1': 'FF Marsh','FFC5': 'FF McKeon','OFC1': 'Capt Mears', 'OFA4': 'Lt Monaghan','OFC3': 'Lt Purcell', \
            'FFA3': 'Prob-FF Preston','OFA1': 'Lt Richardson','FFB2': 'FF Snowling', 'FFC3': 'FF Szerlag', \
            'FFD2': 'FF Stabile', 'OFD3': 'Lt Warner III'}                 


## Dictionary of short names

In [549]:
FF_short_name = {1: 'FF DA', 2: 'FF SA', 3: 'Lt SB', 4: 'Lt TB', 5: 'Lt RB', 6: 'FF AC', \
         7: 'FF MC', 8: 'PFF SC', 9: 'FF AD', 10: 'FF SF', 11: 'Lt RG', 12: 'PFF DG', 13: 'Lt RG', 14: 'Lt MG',  \
         15: 'Lt KH', 16: 'FF MH', 17: 'Lt MJ', 18: 'FF KK', 19: 'FF KL', 20: 'PFF DL', 21: 'FF WM', \
         22: 'Lt EM', 23: 'FF SM', 24: 'Cpt TM', 25: 'Lt MM', 26: 'Cpt KM', 27: 'FF PO', 28: 'PFF JP',  \
         29: 'Lt BP', 30: 'PFF RP', 31: 'Lt WP', 32: 'Lt JR', 33: 'FF GS', 34: 'FF BS', 35: 'FF JS', 36: 'Lt RW'}                 


## Dictionary of initials by Page Number

In [550]:
FF_initials = {1: 'DA', 2: 'SA', 3: 'SB', 4: 'TB', 5: 'RB', 6: 'AC', 7: 'MC', 8: 'SC', 9: 'AD', 10: 'SF', 11: 'RG',  \
         12: 'DG', 13: 'RG', 14: 'MG', 15: 'KH', 16: 'MH', 17: 'MJ', 18: 'KK', 19: 'KL', 20: 'DL', 21: 'WM', \
         22: 'EM', 23: 'SM', 24: 'TM', 25: 'MM', 26: 'KM', 27: 'PO', 28: 'JP', 29: 'WP', 30: 'RP', 31: 'WP', \
         32: 'JR', 33: 'GS', 34: 'BS', 35: 'JS', 36: 'RW'}                 


## Dictionary for looking up platoon by name

In [551]:
FF_platoon = {'FF Andrade': 'C','FF Archambault': 'A','Lt Babcock': 'D','Lt Bailey': 'A','Lt Beaudreau': 'B','FF Campbell': 'A', \
         'FF Columbier': 'B','Prob-FF Crute': 'B','FF DeLuca': 'A','FF Forte': 'C','Lt Gardner': 'B','Prob-FF Gorman': 'D', \
         'Lt Grady': 'B','Lt Greene': 'B','Lt Hall': 'D','FF Howard': 'B','Lt Jones': 'C','FF King': 'C','FF Lang Jr.': 'D', \
         'Prob-FF Lavallee': 'D','FF Marsh': 'D','Lt Matola, Jr.': 'D','FF McKeon': 'C','Capt Mears': 'C','Lt Monaghan': 'A', \
         'Capt Montville': 'A',"FF O'Donnell": 'A','Prob-FF Perry': 'B','Lt Perry': 'B','Prob-FF Preston': 'A',  \
         'Lt Purcell': 'C','Lt Richardson': 'A','FF Snowling': 'B','FF Stabile': 'D','FF Szerlag': 'C','Lt Warner III': 'D'}                 


## Dictionary for looking up dates of employment

In [571]:
FF_de = {'Prob-FF Preston': {'start': datetime.datetime(2017,6,1,0,0,0)}  }               


## Dictionary of 8-day cycle start times by platoon

In [552]:
base_date = {}
base_date['A'] = datetime.datetime(2016,12,26,7,0,0)   #first cycle start 1/1/2017 07:00 or earlier for this platoon
base_date['B'] = datetime.datetime(2016,12,28,7,0,0)
base_date['C'] = datetime.datetime(2016,12,30,7,0,0)
base_date['D'] = datetime.datetime(2017,1,1,7,0,0)

cycle_start = {}                                       # dictionary for cycle start by platoon

max_datetime = datetime.datetime(2018,7,1,0,0,0)

for platoon in base_date.keys():                       # loop through earliest cycle start for each platoon
    cycle_start[platoon] = {}                          # create dictionary for platoon
    ss = base_date[platoon]                            # start with earliest cycle start time
    while (ss < max_datetime):                         # loop until cycle start is past max_datetime
        cycle_start[platoon][ss] = {}                 # add dictionary for cycle start
        ss += datetime.timedelta(days=8)               # increment cycle start by 8 days

cycle_start

print('Number of 8-Day Cycles')
print('A ' + str(len(cycle_start['A'])))
print('B ' + str(len(cycle_start['B'])))
print('C ' + str(len(cycle_start['C'])))
print('D ' + str(len(cycle_start['D'])))

Number of 8-Day Cycles
A 69
B 69
C 69
D 69


## Function checks whether datetime is withing employment period for a FF

In [575]:
def emp_check(ff_name,ts):
    try:
        estart = FF_de[ff_name]['start']
    except KeyError:
        estart = datetime.datetime(2010,1,1,0,0,0)
    try:
        eend = FF_de[ff_name]['end']
    except KeyError:
        eend = datetime.datetime(2030,1,1,0,0,0)
    employed = True
    if (ts < estart): employed = False
    if (ts > eend): employed = False
    return(employed)

## Function generates regularly scheduled shifts for a FF given the cycle start time and date

In [576]:
def add_scheduled_shifts(ff_name,shifts,ts):
    employed = emp_check(ff_name,ts)
    if employed:
        shifts[ff_name]['shifts'][ts] = {}
        shifts[ff_name]['shifts'][ts]['scheduled'] = True
        ts += datetime.timedelta(days=1)   #increment shift start by 1 day
        shifts[ff_name]['shifts'][ts] = {}
        shifts[ff_name]['shifts'][ts]['scheduled'] = True
        ts += datetime.timedelta(days=1)   #increment shift start by 1 day
        ts += datetime.timedelta(hours=10)   #increment shift start 10 hours
        shifts[ff_name]['shifts'][ts] = {}
        shifts[ff_name]['shifts'][ts]['scheduled'] = True
        ts += datetime.timedelta(days=1)   #increment shift start by 1 day
        shifts[ff_name]['shifts'][ts] = {}
        shifts[ff_name]['shifts'][ts]['scheduled'] = True
    return()
    

## Generate dictionary of assigned shifts for each FF

In [577]:
shifts = {}                                          #Dictionary to hold shift by FF data
n_ff = 0                                             #count of FFs
n_shift = 0                                          #cpimt of regular duty shifts

for ff in FF_ID.keys():
    n_ff += 1
    shifts[ff] = {}
    shifts[ff]['platoon'] = FF_platoon[ff]                 #platoon
    shifts[ff]['ff_id'] = FF_ID[ff]                        #id (page number)
    shifts[ff]['ff_desig'] = FF_desig[ff]                  #designation
    shifts[ff]['shifts'] = {}
    
    for shift in cycle_start[shifts[ff]['platoon']]:
        add_scheduled_shifts(ff,shifts,shift)
        n_shift += 4
        
print('Number of Firefighters:')                          # show number of FFs
print(n_ff)
print('Total regular scheduled FF-shifts')                # show total number of shifts, including 'Off' shifts
print(n_shift)

Number of Firefighters:
36
Total regular scheduled FF-shifts
9936


## Read Block FF shift data and store it in the shift dictionary

In [578]:
Page = 0

with open('../Block_FF_shift_data.txt') as f:        #Extract from Block time sheet pdf
    tsheets = f.readlines() 
    for lin in tsheets:                              #Loop through line by line
        words = lin.split()                          #split into words
        if (len(words)) > 1:
            if ('Name:' in words[0]):                #Heading line for new FF
                Page = Page + 1
                ff_name = FF_name[Page]              #look up the name by page number    
            if ('/' in words[0]):                    #first word is mm/dd/yyyy in data lines
                dt = words[0] + ' ' + words[1]
                shift = datetime.datetime.strptime(dt, "%m/%d/%Y %H:%M")   #convert shift start to datetime
                for i in range(2,len(words)):        #loop through the days of the month that appear in the data row
                    try:
                        shifts[ff_name]['shifts'][shift]['Block'] = words[i]   #what Block coded for this day and shift
                    except KeyError:
                        shifts[ff_name]['shifts'][shift] = {}                  #first create dictionary
                        shifts[ff_name]['shifts'][shift]['Block'] = words[i]   #what Block coded for this day and shift
                        
                    n_shift += 1                     
                    shift += datetime.timedelta(days=1)   #increment shift start by 1 day

print('Number of Firefighters:')                          # show number of FFs
print(n_ff)
print('Total FF-shifts (includes "off" shifts)')          # show total number of shifts, including 'Off' shifts
print(n_shift)

Number of Firefighters:
36
Total FF-shifts (includes "off" shifts)
36135


## Add overtime indicators to Block data

These were color coded, black indicating a scheduled shift, red indicating overtime. 

The conversion from .pdf to .txt did not pick up the color coding so this data was entered manually.

In [579]:
n_OT_0700  = 0                                     # initialize counts of OT shifts
n_OT_1700 = 0

with open('../Block_FF_ot.txt') as f:              # open file with OT indicators for Block data
    otime = f.readlines()                          # read line by line
    for lin in otime:                              
        words = lin.split()                        # split the lines into words
        if (len(words) > 0):                       # skip blank lines if any
            Page = int(words[0])                   # first entry is page number
            ff_name = FF_name[Page]                # look up ff_name in dictionary by page number
            month = int(words[1])                  # get year and month for generating datetime of shift
            year = 2017
            if (month < 6): year = 2018
            for i in range(2,len(words),2):        # loop through day and shift codes which indicate OT
                day = int(words[i])            
                if (words[i+1] == '1'):            # set hour=7 for shift start if day shift
                    hour = 7
                    n_OT_0700 += 1                 # increment overtime shift count for day shift
                if (words[i+1]=='2'):              # set hour=17 for shift start if night shift
                    hour = 17
                    n_OT_1700 += 1                 # increment overtime shift count for night shift
                shift = datetime.datetime(year,month,day,hour,0)   # get datetime for shift start
                shifts[ff_name]['shifts'][shift]['Block'] = 'On-OT'          # indicate OT in Block data
                
print('Count of Daytime OT Shifts:')               # show counts of OT shifts (day and night)
print(n_OT_0700)
print('Count of Nightime OT Shifts:')
print(n_OT_1700)

Count of Daytime OT Shifts:
825
Count of Nightime OT Shifts:
960


## Lookup dictionary to get accountability file names by FF

These are the .csv files extracted from Eric's spreadsheet of accountability data

In [580]:
logs = {'FF Andrade': 'FF_Andrade.csv', \
                       'FF Archambault': 'FF_Archambault.csv', \
                       'Lt Babcock': 'Lt_Babcock.csv', \
                       'Lt Bailey': 'Lt_Bailey.csv', \
                       'Lt Beaudreau': 'Lt_Beaudreau.csv', \
                       'FF Campbell': 'FF_Campbell.csv', \
                       'FF Columbier': 'FF_Columbier.csv', \
                       'Prob-FF Crute': 'Prob-FF_Crute.csv', \
                       'FF DeLuca': 'FF_DeLuca.csv', \
                       'FF Forte': 'FF_Forte.csv', \
                       'Lt Gardner': 'Lt_Gardner.csv', \
                       'Prob-FF Gorman': 'Prob-FF_Gorman.csv', \
                       'Lt Grady': 'Lt_Grady.csv', \
                       'Lt Greene': 'Lt_Greene.csv', \
                       'Lt Hall': 'FF_Hall.csv', \
                       'FF Howard': 'FF_Howard.csv', \
                       'Lt Jones': 'Lt_Jones.csv', \
                       'FF King': 'FF_King.csv',
                       'FF Lang Jr.': 'FF_Lang.csv', \
                       'Prob-FF Lavallee': 'Prob-FF_Lavallee.csv', \
                       'FF Marsh': 'FF_Marsh.csv', \
                       'Lt Matola, Jr.': 'Lt_Matola.csv', \
                       'FF McKeon': 'FF_McKeon.csv', \
                       'Capt Mears': 'Capt_Mears.csv', \
                       'Lt Monaghan': 'Lt_Monaghan.csv', \
                       'Capt Montville': 'Capt_Montville.csv', \
                       "FF O'Donnell": 'FF_ODonnell.csv', \
                       'Prob-FF Perry': 'Prob-FF_Perry.csv', \
                       'Lt Perry': 'Lt_Perry.csv', \
                       'Prob-FF Preston': 'Prob-FF_Preston.csv',  \
                       'Lt Purcell': 'Lt_Purcell.csv', \
                       'Lt Richardson': 'Lt_Richardson.csv', \
                       'FF Snowling': 'FF_Snowling.csv', \
                       'FF Stabile': 'FF_Stabile.csv', \
                       'FF Szerlag': 'FF_Szerlag.csv', \
                       'Lt Warner III': 'Lt_Warner.csv'}                 

## Read the accountability logs and add the data to the dictionaries

There is one .csv file containing the accountability data for each firefighter

Data elements are added to the dictionary for that firefighter and shift

In [581]:
def read_acct(ff_name):
    fname = '../' + logs[ff_name]                                   #get the filename for this FF
    acctdf = pd.read_csv(fname,parse_dates=[[0,1]],skiprows=2,header=None)   #read it
    acctdf.rename(columns={'0_1': 'shift',2:'type',3:'hours',4:'rank',5:'reason'},inplace=True)
    acctdf['FF_name'] = ff_name       #create a column for this FFs name
    acctdf['Page'] = FF_ID[ff_name]   #create a column for the page in the Block document
    
    for index, row in acctdf.iterrows():    #loop through row by row and add data to dictionary
        ff_name = row['FF_name']             #shifts dictionary key is FF name
        shift = row['shift']                 #shifts[FF] dictionary key is shift

        try:                                      #create dictionary for accountability data
            shifts[ff_name]['shifts'][shift]['acct'] = {}
        except KeyError:                          #if this shift was not in Block's data add it
            shifts[ff_name]['shifts'][shift] = {}
            shifts[ff_name]['shifts'][shift] = {}
            shifts[ff_name]['shifts'][shift] = {}
            shifts[ff_name]['shifts'][shift]['acct'] = {}
                                                  #fill in the accountability data elements
        shifts[ff_name]['shifts'][shift]['acct']['type'] = row['type']
        shifts[ff_name]['shifts'][shift]['acct']['hours'] = float(row['hours'])
        shifts[ff_name]['shifts'][shift]['acct']['rank'] = row['rank']
        shifts[ff_name]['shifts'][shift]['acct']['reason'] = row['reason']
        rstr = row['reason']                      #determine whose shift was being covered
        for_ff = ''                               #search for specific name in reason string
        if isinstance(rstr, str):
            if ('Andrade' in rstr):  for_ff = 'FF Andrade'
            if ('Archambault' in rstr):  for_ff = 'FF Archambault'
            if ('Babcock' in rstr):  for_ff = 'Lt Babcock'
            if ('Bailey' in rstr): for_ff = 'Lt Bailey'
            if ('Beaudreau' in rstr): for_ff = 'Lt Beaudreau'
            if ('Campbell' in rstr): for_ff = 'FF Campbell'
            if ('Columbier' in rstr): for_ff = 'FF Columbier'
            if ('Crute' in rstr): for_ff = 'Prob-FF Crute'
            if ('DeLuca' in rstr): for_ff = 'FF DeLuca'
            if ('Forte' in rstr): for_ff = 'FF Forte'
            if ('Gardner' in rstr): for_ff = 'Lt Gardner'
            if ('Gorman' in rstr): for_ff = 'Prob-FF Gorman'
            if ('Grady' in rstr): for_ff = 'Lt Grady'
            if ('Greene' in rstr): for_ff = 'Lt Greene'
            if ('Hall' in rstr): for_ff = 'Lt Hall'
            if ('Howard' in rstr): for_ff = 'FF Howard'
            if ('Jones' in rstr): for_ff = 'Lt Jones'
            if ('King' in rstr): for_ff = 'FF King'
            if ('Lang' in rstr): for_ff = 'FF Lang Jr.'
            if ('Lavallee' in rstr): for_ff = 'Prob-FF Lavallee'
            if ('Marsh' in rstr): for_ff = 'FF Marsh'
            if ('Matola' in rstr): for_ff = 'Lt Matola, Jr.'
            if ('McKeon' in rstr): for_ff = 'FF McKeon'
            if ('Mears' in rstr): for_ff = 'Capt Mears'
            if ('Monaghan' in rstr): for_ff = 'Lt Monaghan'
            if ('Montville' in rstr): for_ff = 'Capt Montville'
            if ('Donnell' in rstr): for_ff = "FF O'Donnell"
            if ('FF Perry' in rstr): for_ff = 'Prob-FF Perry'
            if ('Lt Perry' in rstr): for_ff = 'Lt Perry'
            if ('Preston' in rstr): for_ff = 'Prob-FF Preston'
            if ('Purcell' in rstr): for_ff = 'Lt Purcell'
            if ('Richardson' in rstr): for_ff = 'Lt Richardson'
            if ('Snowling' in rstr): for_ff = 'FF Snowling'
            if ('Stabile' in rstr): for_ff = 'FF Stabile'
            if ('Szerlag' in rstr): for_ff = 'FF Szerlag'
            if ('Warner' in rstr): for_ff = 'Lt Warner III'
            shifts[ff_name]['shifts'][shift]['acct']['for_ff'] = for_ff    #save FF being covered
            for_id=''
            if (len(for_ff) > 0):
                for_id = FF_ID[for_ff]
                shifts[ff_name]['shifts'][shift]['acct']['for_id'] = for_id  #save page number in Block
    return()

In [582]:
for ff in logs.keys():
    print(ff)
    read_acct(ff)

FF Andrade
FF Archambault
Lt Babcock
Lt Bailey
Lt Beaudreau
FF Campbell
FF Columbier
Prob-FF Crute
FF DeLuca
FF Forte
Lt Gardner
Prob-FF Gorman
Lt Grady
Lt Greene
Lt Hall
FF Howard
Lt Jones
FF King
FF Lang Jr.
Prob-FF Lavallee
FF Marsh
Lt Matola, Jr.
FF McKeon
Capt Mears
Lt Monaghan
Capt Montville
FF O'Donnell
Prob-FF Perry
Lt Perry
Prob-FF Preston
Lt Purcell
Lt Richardson
FF Snowling
FF Stabile
FF Szerlag
Lt Warner III


In [584]:
shifts['Prob-FF Preston']

{'ff_desig': 'FFA3',
 'ff_id': 30,
 'platoon': 'A',
 'shifts': {datetime.datetime(2017, 6, 1, 7, 0): {'Block': 'Off'},
  datetime.datetime(2017, 6, 1, 17, 0): {'Block': 'Off'},
  datetime.datetime(2017, 6, 2, 7, 0): {'Block': 'Off'},
  datetime.datetime(2017, 6, 2, 17, 0): {'Block': 'Off'},
  datetime.datetime(2017, 6, 3, 7, 0): {'Block': 'Off'},
  datetime.datetime(2017, 6, 3, 17, 0): {'Block': 'Off'},
  datetime.datetime(2017, 6, 4, 7, 0): {'Block': 'On', 'scheduled': True},
  datetime.datetime(2017, 6, 4, 17, 0): {'Block': 'Off'},
  datetime.datetime(2017, 6, 5, 7, 0): {'Block': 'On', 'scheduled': True},
  datetime.datetime(2017, 6, 5, 17, 0): {'Block': 'Off'},
  datetime.datetime(2017, 6, 6, 7, 0): {'Block': 'Off'},
  datetime.datetime(2017, 6, 6, 17, 0): {'Block': 'On', 'scheduled': True},
  datetime.datetime(2017, 6, 7, 7, 0): {'Block': 'Off'},
  datetime.datetime(2017, 6, 7, 17, 0): {'Block': 'On', 'scheduled': True},
  datetime.datetime(2017, 6, 8, 7, 0): {'Block': 'Off'},
  da

## Determine how much Block overstated OT due to assumption of full shifts only

In [561]:
Block_hours = {}
accountability_hours = {}

hours_Block = 0.0
hours_acct  = 0.0

tot_hours_Block = 0.0
tot_hours_acct  = 0.0

discrepancies = 0

for ff in shifts.keys():                #loop through FFs
    hours_Block = 0.0                   # initialize OT total for Block
    hours_acct  = 0.0                   # initialize OT total for accountability logs
    
    print('\n' + ff + '\n')             # show FF
    
    for shift in shifts[ff]['shifts'].keys():                               #loop through shifts for FF
        if (('Block' in shifts[ff]['shifts'][shift].keys()) &  \
            ('acct' in shifts[ff]['shifts'][shift].keys())):                #only if in Block data and acct
            if(shifts[ff]['shifts'][shift]['Block']=='On-OT'):              # flagged as OT by Block
                shift_hours = 10.0                                          # shift hours is 10 for day shift
                if (shift.hour==17): shift_hours = 14.0                     # or 14 for night shift
                hours_Block += shift_hours                                  # add nunmber of hours in shift total
                acct_hours = shifts[ff]['shifts'][shift]['acct']['hours']             # get accountability log hours
                hours_acct += acct_hours                                    # add them to total accountability hours
                if(acct_hours < shift_hours):                               # count a discrepancy if acct is smaller
                    discrepancies += 1
                    print(shift)                                            # display the shift and difference
                    print(str(acct_hours) + " versus " + str(shift_hours))
                    print(shifts[ff]['shifts'][shift]['acct']['reason'])              # show the reason from acct log
    print('\n FF OT hours (Block)\n')                                       # show Block total OT hours for this FF
    print(hours_Block)                                                      
    print('\n FF OT hours (logs)\n')                                        # show acct log total OT hours for this FF
    print(hours_acct)
    tot_hours_Block += hours_Block                                          # add hours for this FF to totals
    tot_hours_acct  += hours_acct
    
print('\n Total OT hours (Block)\n')                                        # show total OT hours for Block
print(tot_hours_Block)
print('\n Total OT hours (logs)\n')                                         # show total OT hours for acct log
print(tot_hours_acct)
print('\n Difference\n')                                                    # show the difference
diff = tot_hours_Block - tot_hours_acct
print(diff)

print('\n Number of partial shifts \n')                                     # show count of partial shifts 
print(discrepancies)


FF Andrade

2017-10-22 17:00:00
12.0 versus 14.0
ORDERED IN TO FILL REST OF SHIFT FOR R1
2017-11-14 07:00:00
8.0 versus 10.0
AFG TRAINING
2018-04-11 07:00:00
0.5 versus 10.0
#1195

 FF OT hours (Block)

286.0

 FF OT hours (logs)

272.5

FF Archambault

2017-06-20 17:00:00
0.5 versus 14.0
LATE CALL 1777
2017-08-07 17:00:00
0.5 versus 14.0
I#2351
2017-09-08 17:00:00
1.0 versus 14.0
2754
2017-11-15 07:00:00
8.0 versus 10.0
AFG TRAINING
2018-03-23 07:00:00
1.0 versus 10.0
FF Andrade, Daniel - Sick

 FF OT hours (Block)

232.0

 FF OT hours (logs)

181.0

Lt Babcock

2017-06-30 07:00:00
4.0 versus 10.0
Lt Perry, William - Bargaining
2017-06-02 17:00:00
0.5 versus 14.0
HOLDOVER #1553
2017-07-01 17:00:00
1.5 versus 14.0
Late Relief
2017-07-20 17:00:00
1.5 versus 14.0
awaiting relief from R2 officer
2017-08-24 07:00:00
1.0 versus 10.0
[#2579
2017-08-06 17:00:00
4.0 versus 14.0
#2324 DIVETEAM
2017-08-10 17:00:00
1.5 versus 14.0
l # 2409
2017-08-13 17:00:00
2.5 versus 14.0
holdover #2435
2017-

Held Over On Run # 3755
2017-11-04 17:00:00
0.25 versus 14.0
l#3476
2017-12-05 17:00:00
1.0 versus 14.0
Late Run |#17-3807/3808
2017-12-09 17:00:00
1.0 versus 14.0
i#3842
2018-01-06 17:00:00
9.5 versus 14.0
Dispatcher Collins Jr, Richard - Dispatch night
2018-02-08 17:00:00
0.5 versus 14.0
late ﬁre call#500
2018-03-26 17:00:00
12.0 versus 14.0
Capt Mears, Thomas (D) - Injured On Duty

 FF OT hours (Block)

480.0

 FF OT hours (logs)

376.75

FF O'Donnell

2017-08-07 17:00:00
0.5 versus 14.0
I# 2351
2017-09-08 17:00:00
1.0 versus 14.0
2754
2017-11-15 07:00:00
8.0 versus 10.0
AFG TRAINING
2017-12-05 17:00:00
0.5 versus 14.0
Late Run I#17-3807
2017-12-06 17:00:00
0.5 versus 14.0
Late Run I#17-3821

 FF OT hours (Block)

76.0

 FF OT hours (logs)

20.5

Prob-FF Perry

2017-06-23 17:00:00
1.0 versus 14.0
Late Call # 1817
2017-06-27 17:00:00
7.0 versus 14.0
FF McKeon, Steven Sick
2017-12-28 17:00:00
1.0 versus 14.0
Held Over on Run # 4092
2018-01-04 17:00:00
1.0 versus 14.0
Late run |#18-005

## Find shifts for which Block coded 'OT' but no accountability record matches

In [562]:
unmatched = 0

for ff in shifts.keys():
    
    print('\n' + ff + '\n')
    
    for shift in shifts[ff]['shifts'].keys():
        if (('Block' in shifts[ff]['shifts'][shift].keys()) & ('acct' not in shifts[ff]['shifts'][shift].keys())):
            if(shifts[ff]['shifts'][shift]['Block']=='On-OT'):
                print(shift);
                unmatched += 1

print('\n Unmatched \n')
print(unmatched)


FF Andrade


FF Archambault


Lt Babcock


Lt Bailey


Lt Beaudreau


FF Campbell

2018-05-13 17:00:00

FF Columbier


Prob-FF Crute

2018-05-05 17:00:00
2018-05-21 17:00:00

FF DeLuca


FF Forte


Lt Gardner


Prob-FF Gorman

2018-05-04 17:00:00

Lt Grady

2018-05-12 17:00:00

Lt Greene

2018-05-01 17:00:00
2018-05-05 17:00:00

Lt Hall

2018-05-06 17:00:00
2018-05-09 17:00:00
2018-05-10 17:00:00
2018-05-18 17:00:00
2018-05-19 17:00:00

FF Howard

2018-05-05 17:00:00

Lt Jones


FF King


FF Lang Jr.

2018-05-02 17:00:00
2018-05-09 17:00:00
2018-05-11 17:00:00
2018-05-17 17:00:00

Prob-FF Lavallee


FF Marsh

2017-08-09 07:00:00
2017-08-22 17:00:00
2017-12-27 17:00:00
2018-01-04 17:00:00

Lt Matola, Jr.

2018-05-09 07:00:00
2018-05-05 17:00:00

FF McKeon


Capt Mears

2018-05-01 17:00:00

Lt Monaghan


Capt Montville

2018-05-07 17:00:00

FF O'Donnell


Prob-FF Perry


Lt Perry

2018-01-31 17:00:00
2018-05-01 17:00:00
2018-05-17 17:00:00

Prob-FF Preston

2018-05-12 17:00:00
2018-05-2

## Find shifts for which Block did not code 'OT' but accountability record indicates OT

In [563]:
undetected = 0

for ff in shifts.keys():
    
    print('\n' + ff + '\n')
    
    for shift in shifts[ff]['shifts'].keys():
        if (('Block' in shifts[ff]['shifts'][shift].keys()) & ('acct' in shifts[ff]['shifts'][shift].keys())):
            if((shifts[ff]['shifts'][shift]['Block']!='On-OT') & ('OT' in shifts[ff]['shifts'][shift]['acct']['type'])):
                print(shift)
                print(shifts[ff]['shifts'][shift])
                undetected += 1

print('\n Undetected \n')
print(undetected)


FF Andrade


FF Archambault


Lt Babcock

2018-05-13 17:00:00
{'Block': 'Off', 'acct': {'type': 'OT-IOD', 'hours': 14.0, 'rank': 'Lt', 'reason': 'Capt Mears, Thomas (D) - Injured On Duty', 'for_ff': 'Capt Mears', 'for_id': 24}}

Lt Bailey

2018-05-11 17:00:00
{'Block': 'Off', 'acct': {'type': 'OT-ADM', 'hours': 14.0, 'rank': 'Lt', 'reason': 'Lt Grady, Ryan (D) - Administrative - Military Leave', 'for_ff': 'Lt Grady', 'for_id': 13}}

Lt Beaudreau

2018-05-04 17:00:00
{'Block': 'Off', 'acct': {'type': 'OT-IOD', 'hours': 14.0, 'rank': 'Lt', 'reason': 'Lt Purcell, William (D) - Injured On Duty', 'for_ff': 'Lt Purcell', 'for_id': 31}}
2018-05-13 17:00:00
{'Block': 'Off', 'acct': {'type': 'OT-IOD', 'hours': 14.0, 'rank': 'Lt', 'reason': 'Lt Purcell, William (D) - Injured On Duty', 'for_ff': 'Lt Purcell', 'for_id': 31}}
2018-05-20 17:00:00
{'Block': 'Off', 'acct': {'type': 'OT-IOD', 'hours': 14.0, 'rank': 'Lt', 'reason': 'Lt Purcell, William (D) - Injured On Duty', 'for_ff': 'Lt Purcell', 'f

## Write web page displaying results

In [564]:
def shiftabl(year,month,hour,shifts,file,ff_name):        
    ndays = {1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
    ff_desig = FF_desig[ff_name]
    if (hour == 7): 
        full_shift=10
        file.write('<tr align="center"><td rowspan="2">' + str(month) + '/' + str(year) + '</td>')
        file.write('<td>07:00</td>')
    else: 
        full_shift=14
        file.write('<tr><td>17:00</td>')
    
    for d in range(1,1+ndays[month]):
        ts = datetime.datetime(year,month,d,hour,0,0)
        tag = ff_desig + str(ts)                        # tag for link into accountability log
        try:                  
            sdict = shifts[ff_name]['shifts'][ts]       # see if there is an entry for this shift start 
            tds = '<td>'                                # if there is, open a cell tag
            try:                                        # first check for partial shift
                hours = sdict['acct']['hours']          # set background red for partial shifts
                if (hours < full_shift):
                    tds = '<td bgcolor="#FF0000">'
                typ = sdict['acct']['type']             # set background colors for other situations
                if (('C-' in typ) | (typ == 'DP') | ('CS-' in typ)):# set background green for collateral duty
                    tds = '<td bgcolor="#00FF00">'
                if (typ == 'IOD'):                      # set background purple for IOD
                    tds = '<td bgcolor="#FF00FF">'
                if (typ == 'SL'):                       # set background gray for SL
                    tds = '<td bgcolor="#C0C0C0">'
                if (typ == 'VC'):                       # set background gray for SL
                    tds = '<td bgcolor="#00FFFF">'
                if ((typ == 'PRS') | (typ == 'BER')):   # set background yellow for personal and bereavement 
                    tds = '<td bgcolor="#FFFF00">'
            except KeyError:
                tds = '<td>'

            file.write(tds)
            try:                                                    #look for Block's code
                block_type = sdict['Block']
                if (block_type == 'On-OT'): block_type = 'OT'       #display 'OT' for overtime
                if (block_type in ['Off']): block_type = '&nbsp;'   #display blanks for 'Off'
                file.write(block_type)
            except KeyError:
                file.write('&nbsp;')
            file.write('<br>')
            try:                                                    #look for accountability log data
                acct = sdict['acct']
                file.write('<a href="#' + tag + '">' + acct['type'] + '</a>')
            except KeyError:
                file.write('&nbsp;')
            file.write('<br>')
            try:                                                    #see who we are covering for (if anyone)
                for_ff = sdict['acct']['for_ff']
                if (len(for_ff) > 1):
                    for_desig = FF_desig[for_ff]
                    for_ff_tag = for_desig + str(ts)
                    file.write('<a href="#' + for_ff_tag + '">' + for_desig + '</a>')
                else:                                               # if we're not covering, check to see if anyone 
                    for cov_ff_id in range(1,37):                   # is covering us
                        cov_ff_name = FF_name[cov_ff_id]
                        cov_ff_desig = FF_desig[cov_ff_name]
                        try:
                            cov_acct = shifts[cov_ff_name]['shifts'][ts]['acct']
                            if (cov_acct['for_ff'] == ff_name):
                                cov_tag = cov_ff_desig + str(ts)
                                file.write('<a href="#' + cov_tag + '">' + cov_ff_desig + '</a>')
                        except KeyError:
                            continue
                    file.write('&nbsp;')
            except KeyError:
                file.write('&nbsp;')
            file.write('</td>')
        except KeyError:
            file.write('<td>Err</td>')
    file.write('</tr>\n')
    return()

In [565]:
file = open('../test.html','w') 

file.write('<html><body>') 

for ff_id in range(1,37):
    ff_name = FF_name[ff_id]
    ff_desig = FF_desig[ff_name]
    file.write('<a name="' + ff_desig + '">')
    file.write('<h2>' + ff_desig + ' (' + ff_name + ')'+ '</h2></a>\n')
    file.write('<table border="1">\n')
    
    file.write('<tr><td>Month</td><td>Shift</td>')
    for d in range(1,32):
        file.write('<td>' + str(d) + '</td>')
    file.write('</tr>\n')
    
    for month in [6,7,8,9,10,11,12,1,2,3,4,5]:
        if (month < 6): yr = 2018
        else: yr = 2017 
            
        shiftabl(yr,month,7,shifts,file,ff_name)  
        shiftabl(yr,month,17,shifts,file,ff_name) 
        
    file.write('</table>\n')
    
for ff_id in range(1,37):
    ff_name = FF_name[ff_id]
    ff_desig = FF_desig[ff_name]
    file.write('<a name="acct_' + ff_desig + '">\n')
    file.write('<h2>' + ff_desig + ' (' + ff_name + ')' + '</h2></a>\n')
    file.write('<table border="1">\n')
    file.write('<tr><td>Shift</td><td>Type</td><td>Hours</td><td>Rank</td><td>Reason</td></tr>\n')

    for shift in shifts[ff_name]['shifts'].keys():
        try:
            entry_type = shifts[ff_name]['shifts'][shift]['acct']['type']
            hours = str(shifts[ff_name]['shifts'][shift]['acct']['hours'])
            rank = shifts[ff_name]['shifts'][shift]['acct']['rank']
            reason = str(shifts[ff_name]['shifts'][shift]['acct']['reason'])
            tag = ff_desig + str(shift)
            file.write('<tr><td><a name="' + tag + '"></a>' + str(shift) + '</td>')
            file.write('<td>' + entry_type + '</td>')
            file.write('<td>' + hours + '</td>')
            file.write('<td>' + rank + '</td>')
            file.write('<td>' + reason + '</td></tr>\n')
        except KeyError:
            continue
    file.write('</table>\n')

file.close() 

## Check for situation of alleged 'gaming' OT and VAC

Scenario:  FF1 takes a vacation day, FF2 gets OT for covering FF1's vacation day
           Later in FF1's 8 day cycle, FF2 takes a vacation day and FF1 gets OT for covering it.
           
Algorithm: 

            For each FF, look at 8-day cycles in which he took a vacation day (or other day requiring coverage).  

            See if, in this 8-day cycle, this FF was paid for covering a day taken by the FF who covered his. 

In [566]:
match_count = 0

for ff_name in shifts.keys():                          # loop through FF list
    platoon = shifts[ff_name]['platoon']               # get the platoon (A,B,C,D) for this FF
    ff_id = shifts[ff_name]['ff_id']                   # get page number
    print('Processing ' + ff_name + ' ' + str(ff_id) + ' ' + platoon)           # display name and id #
    for first_shift in cycle_start[platoon]:           # loop through shift start times that begin an 8-day cycle
        cycle_shifts = []                              # fill in the other shift start times for that cycle
        ts = first_shift
        for day in range(1,9):
            cycle_shifts.append(ts)
            ts += datetime.timedelta(hours=10)          
            cycle_shifts.append(ts)
            ts += datetime.timedelta(hours=14)
        for ts in cycle_shifts:                                      # loop through the shifts for this 8-day cycle
            try:
                stype = shifts[ff_name]['shifts'][ts]['acct']['type']   # get accountability log entry if there is one
                if (stype in ['VC','SL','PRS','ADM','BER','ACT']):      # see if coverage may be required
                    print(ff_name + ' ' + str(ts) + ' ' + stype)    
                    for ff_cover in shifts.keys():                   # see who covered this shift
                        if (ff_cover != 'ff_name'):
                            try:
                                if (shifts[ff_cover]['shifts'][ts]['acct']['for_id'] == ff_id):
                                    print('covered by: ' + ff_cover )
                                    cover_id = FF_ID[ff_cover]
                                    for cs in cycle_shifts:          # see if the ff who covered was himself covered
                                        try:                         # by the ff we are processing in this cycle
                                            if (shifts[ff_name]['shifts'][cs]['acct']['for_id'] == cover_id):
                                                typ = shifts[ff_name]['shifts'][cs]['acct']['type']
                                                print('match########## ' + str(cs) + ' ' + ff_name + ' for ' + ff_cover + ' ' + typ)
                                                if ('SWAP' not in typ): match_count += 1
                                        except KeyError:
                                            continue
                            except KeyError:
                                continue
            except KeyError:
                continue                                             #get here if no accountability log
                                
print(match_count)        

Processing FF Andrade 1 C
FF Andrade 2017-01-10 17:00:00 SL
FF Andrade 2017-02-09 07:00:00 SL
FF Andrade 2017-02-18 17:00:00 VC
covered by: Prob-FF Perry
FF Andrade 2017-04-05 07:00:00 SL
covered by: Prob-FF Lavallee
FF Andrade 2017-04-24 17:00:00 VC
FF Andrade 2017-04-29 07:00:00 SL
FF Andrade 2017-04-30 07:00:00 SL
covered by: Prob-FF Perry
FF Andrade 2017-05-01 17:00:00 SL
covered by: FF Columbier
FF Andrade 2017-05-02 17:00:00 SL
covered by: Prob-FF Gorman
FF Andrade 2017-05-07 07:00:00 VC
FF Andrade 2017-06-26 17:00:00 PRS
covered by: FF Snowling
FF Andrade 2017-07-03 07:00:00 VC
covered by: Prob-FF Crute
FF Andrade 2017-09-29 07:00:00 BER
covered by: Prob-FF Preston
FF Andrade 2017-09-30 17:00:00 BER
FF Andrade 2017-10-01 17:00:00 BER
FF Andrade 2017-10-06 07:00:00 ACT
FF Andrade 2017-10-07 07:00:00 VC
covered by: FF Columbier
FF Andrade 2017-10-15 07:00:00 VC
covered by: FF Columbier
FF Andrade 2017-10-23 07:00:00 VC
FF Andrade 2017-10-25 17:00:00 VC
covered by: FF Columbier
FF 

Lt Matola, Jr. 2017-12-19 07:00:00 SL
covered by: Lt Greene
Lt Matola, Jr. 2017-12-20 07:00:00 SL
covered by: Capt Mears
Lt Matola, Jr. 2017-12-21 17:00:00 SL
covered by: Lt Perry
Lt Matola, Jr. 2017-12-22 17:00:00 SL
covered by: Lt Gardner
Lt Matola, Jr. 2017-12-27 07:00:00 SL
covered by: Lt Greene
Lt Matola, Jr. 2017-12-28 07:00:00 SL
covered by: Capt Mears
Lt Matola, Jr. 2017-12-29 17:00:00 SL
covered by: Lt Perry
Lt Matola, Jr. 2017-12-30 17:00:00 SL
covered by: Lt Monaghan
Lt Matola, Jr. 2018-01-04 07:00:00 SL
covered by: FF King
Lt Matola, Jr. 2018-01-05 07:00:00 SL
covered by: Lt Jones
covered by: FF McKeon
Lt Matola, Jr. 2018-01-06 17:00:00 SL
Lt Matola, Jr. 2018-01-07 17:00:00 SL
Lt Matola, Jr. 2018-01-12 07:00:00 SL
Lt Matola, Jr. 2018-01-13 07:00:00 SL
Lt Matola, Jr. 2018-01-14 17:00:00 SL
covered by: Lt Perry
Lt Matola, Jr. 2018-01-15 17:00:00 SL
Lt Matola, Jr. 2018-01-20 07:00:00 SL
Lt Matola, Jr. 2018-01-21 07:00:00 SL
covered by: Lt Gardner
Lt Matola, Jr. 2018-01-22 17:0

In [567]:
def ptabl(platoon,ts,shifts,file):        
    file.write('<tr><td>' + str(ts) + '</td><td>' + platoon + '</td>')
    for i in range(1,5):
        ff_desig = 'OF' + platoon + str(i)
        ff_name = desig_FF[ff_desig]
        tds =('<td>')
        try:
            typ = shifts[ff_name]['shifts'][ts]['acct']['type']
            if (typ == 'IOD'):                      # set background purple for IOD
                tds = '<td bgcolor="#FF00FF">'
            if (typ == 'SL'):                       # set background gray for SL
                tds = '<td bgcolor="#C0C0C0">'
            if (typ == 'VC'):                       # set background light blue for VC
                tds = '<td bgcolor="#00FFFF">'
            if ((typ == 'PRS') | (typ == 'BER')):   # set background yellow for personal and bereavement 
                tds = '<td bgcolor="#FFFF00">'
            file.write(tds + typ)
        except KeyError:
            file.write(tds + 'On')
        file.write('</td>')
    for i in range(1,6):
        ff_desig = 'FF' + platoon + str(i)
        ff_name = desig_FF[ff_desig]
        tds =('<td>')
        try:
            typ = shifts[ff_name]['shifts'][ts]['acct']['type']
            if (typ == 'IOD'):                      # set background purple for IOD
                tds = '<td bgcolor="#FF00FF">'
            if (typ == 'SL'):                       # set background gray for SL
                tds = '<td bgcolor="#C0C0C0">'
            if (typ == 'VC'):                       # set background light blue for VC
                tds = '<td bgcolor="#00FFFF">'
            if ((typ == 'PRS') | (typ == 'BER')):   # set background yellow for personal and bereavement 
                tds = '<td bgcolor="#FFFF00">'
            file.write(tds + typ)
        except KeyError:
            if ((ff_name == 'Prob-FF Preston') & (ts < datetime.datetime(2017,6,1,0,0,0))):
                file.write(tds + 'NA')
            else:
                file.write(tds + 'On')
        file.write('</td>')
    file.write('</tr>\n')
    return()

In [568]:
file = open('../platoons.html','w') 

file.write('<html><body>') 

for platoon in ['A','B','C','D']:
    file.write('<h2>Platoon: ' + platoon + '</h2></a>\n')
    file.write('<table border="1">\n')
    file.write('<tr><td>&nbsp;</td><td>&nbsp;</td>')
    for i in range(1,5):
        file.write('<td>OF' + platoon + str(i) + '</td>')
    for i in range(1,6):
        file.write('<td>FF' + platoon + str(i) + '</td>')
    file.write('</tr>\n')
    file.write('<tr><td>Shift</td><td>P</td>')
    for i in range(1,5):
        des = 'OF' + platoon + str(i)
        file.write('<td>'+ desig_FF[des] + '</td>')
    for i in range(1,6):
        des = 'FF' + platoon + str(i)
        file.write('<td>' + desig_FF[des]+ '</td>')
    file.write('</tr>\n')
        
    for ts in cycle_start[platoon]:
        if (ts < datetime.datetime(2018,5,21,1,0,0)):
            ts2 = ts
            ptabl(platoon,ts2,shifts,file)
            ts2 += datetime.timedelta(days=1)   #increment shift start by 1 day
            ptabl(platoon,ts2,shifts,file)
            ts2 += datetime.timedelta(days=1)   #increment shift start by 1 day
            ts2 += datetime.timedelta(hours=10)   #increment shift start 10 hours
            ptabl(platoon,ts2,shifts,file)
            ts2 += datetime.timedelta(days=1)   #increment shift start by 1 day
            ptabl(platoon,ts2,shifts,file)
    file.write('</table>\n')

file.close() 